In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

In [8]:
tokenizer = Tokenizer()
data = open('House.txt',encoding="utf8").read()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [9]:
input_sequences = []
for line in corpus:
 token_list = tokenizer.texts_to_sequences([line])[0]
 for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  input_sequences.append(n_gram_sequence)
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [10]:
model = Sequential()
model.add(Embedding(total_words, 20, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 181, 20)           28940     
_________________________________________________________________
bidirectional (Bidirectional (None, 181, 300)          205200    
_________________________________________________________________
dropout (Dropout)            (None, 181, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 723)               73023     
_________________________________________________________________
dense_1 (Dense)              (None, 1447)              1047628   
Total params: 1,515,191
Trainable params: 1,515,191
Non-trainable params: 0
____________________________________________

In [ ]:
history = model.fit(predictors, label, epochs=50, verbose=1)

Train on 6444 samples
Epoch 1/50
6444/6444 [==============================] - 89s 14ms/sample - loss: 6.8430 - accuracy: 0.0334
Epoch 2/50
6444/6444 [==============================] - 94s 15ms/sample - loss: 6.1294 - accuracy: 0.0362
Epoch 3/50
6444/6444 [==============================] - 91s 14ms/sample - loss: 6.0280 - accuracy: 0.0386
Epoch 4/50
6444/6444 [==============================] - 90s 14ms/sample - loss: 5.9471 - accuracy: 0.0461
Epoch 5/50
6444/6444 [==============================] - 89s 14ms/sample - loss: 5.8453 - accuracy: 0.0525
Epoch 6/50
6444/6444 [==============================] - 93s 14ms/sample - loss: 5.7573 - accuracy: 0.0551
Epoch 7/50
6444/6444 [==============================] - 91s 14ms/sample - loss: 5.6887 - accuracy: 0.0601
Epoch 8/50
2400/6444 [==========>...................] - ETA: 53s - loss: 5.6667 - accuracy: 0.0567

In [58]:
seed_text = "Fight"
next_words = 10
  
for _ in range(next_words):
 token_list = tokenizer.texts_to_sequences([seed_text])[0]
 token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
 predicted = model.predict_classes(token_list, verbose=0)
 output_word = ""
 for word, index in tokenizer.word_index.items():
  if index == predicted:
   output_word = word
   break
 seed_text += " " + output_word
print(seed_text)


Fight there were texans with the immobile staring still set the
